# MRN Normalization

1.  Loading data into a gene expression matrix.
2.  Munging data
3.  MRN Normalization
4.  Quick shot at maching learning.

"Trimmed Mean of M-values" (TMM) normalization, published by [Robinson and Oshlack](https://www.frontiersin.org/articles/10.3389/fgene.2016.00164/full#B16) is a widely used method of normalizing gene expression in scRNA data.  A variant called MRN (Median Ratio Normalization) is described by [Maza et al.](https://www.tandfonline.com/doi/full/10.4161/cib.25849), and may perform slightly better than TMM.  We carry out MRN normalization on the P1902 scRNA data, and use the results in a quick machine learning application.

In [1]:
#Import packages.  Put plots "inline" in the notebook.  

import numpy as np  # For numerical computations.
import pandas as pd  # Pandas for data analysis.
import matplotlib.pyplot as plt  # For basic plotting.
import seaborn as sns # For pretty visualization in Seaborn.  See https://seaborn.pydata.org/

import os # Working with file directories, etc.

from IPython.display import display # Pretty display of data frames.

# Put plots inline rather than in a pop-up.
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #got annoyed of these red boxes

Go to section 2, if you have already loaded the data and have the pickles.

# 1.  Loading the data

Start at section 2, if the raw data has already been processed and pickled.

In this section, we load the gene expression data and some metadata.  The sequencing data is stored in a 'merged_gene_counts.txt'.  The metadata is stored in a separate file called 'meta_data_marty_inVitro_feb8.csv'.  We use the metadata to select bulk-cells from  experiment P9855.  The first function loads the metadata.

In [2]:
def get_meta(experiment, filename = 'meta_data_marty_inVitro_feb8.csv', report=True, bulks = False):
    df = pd.read_csv(filename, sep=',',
                     index_col=2, header=0, low_memory=False)
    df = df[ df['Project_ID'] == experiment] # Only cells from the experiment.
    if bulks:
        df = df[ df['Number_Of_Cells'] >= 25] # Include only bulks.
    else:
        df = df[ df['Number_Of_Cells'] == 1] # Get rid of bulks.
    if report:
        if bulks:
            print('{} bulks found in experiment {}'.format(len(df), experiment))
        else:
            print('{} single cells found in experiment {}'.format(len(df), experiment))
        clones = df['Clone_ID'].unique()
        print('{} Clones: {}'.format(len(clones), ', '.join(clones)))
        print('The first five rows of the dataframe are below')
        display(df[:5])
    return df

In [3]:
meta_df = get_meta('P9855', filename = 'meta_data_marty_inVitro_feb8.csv')

0 single cells found in experiment P9855
0 Clones: 
The first five rows of the dataframe are below


,Project_ID,Donor,Number_Of_Cells,Clone_ID,In_Vivo_Clone_ID
Sample_ID,,,,,


In [5]:
meta_bulks = get_meta('P9855', filename = 'meta_data_marty_inVitro_feb8.csv', bulks=True)

99 bulks found in experiment P9855
53 Clones: P1_A03, P1_A05, P1_A12, P1_D09, P1_D12, P1_F10, P1_H01, P1_H10, P1_A08, P1_B10, P1_C11, P1_F02, P1_F04, P2_A04, P2_B11, P2_D12, P2_F01, P2_G01, P2_G09, P2_H01, P2_H04, P2_H05, P2_A05, P2_C12, P1_A03_ATAC_wellA1_rep1_1000, P1_A05_ATAC_wellA3_705, P1_D09_ATAC_wellA5_410, P1_F10_ATAC_wellA8_1000, P1_H01_ATAC_wellA9_rep1_1000, P1_H01_ATAC_wellA10_rep2_1000, P1_H10_ATAC_wellA11_rep1_1000, P1_A08_ATAC_wellB1_350, P1_A03_ATAC_wellA2_rep2_1000, P1_A12_ATAC_wellA4_506, P1_D12_ATAC_wellA6_1000, P1_H10_ATAC_wellA12_rep2_1000, P1_B10_ATAC_wellB2_269, P1_C11_wellB3_500, P1_F02_wellB4_500, P1_F04_wellB5_500, P2_A04_wellC1_1000_rep1, P2_A04_wellC2_1000_rep2, P2_B11_wellC3_700, P2_H05_wellC5_1000, P2_F01_wellC6_1000, P2_D12_wellC7_1000, P2_G01_wellC8_1000, P2_G09_wellC9_918, P2_H01_wellC10_1000, P2_H04_wellC11_1000_rep1, P2_H04_wellC12_1000_rep2, P2_H05_wellD1_1000, P2_A05_wellD2_1000
The first five rows of the dataframe are below


,Project_ID,Donor,Number_Of_Cells,Clone_ID,In_Vivo_Clone_ID
Sample_ID,,,,,
P9855_2001,P9855,YFV2003,25,P1_A03,NaN
P9855_2002,P9855,YFV2003,25,P1_A03,NaN
P9855_2003,P9855,YFV2003,25,P1_A03,NaN
P9855_2004,P9855,YFV2003,25,P1_A05,NaN
P9855_2005,P9855,YFV2003,25,P1_A05,NaN


In [6]:
meta_bulks.drop(meta_bulks.index[[range(69,99)]], inplace=True) # drop atac data... and c12 while we are cutting
                                                                # the tail off of the dataframe 
meta_bulks.drop(['In_Vivo_Clone_ID'],axis=1, inplace=True)
meta_bulks['Clone_ID'] = meta_bulks['Clone_ID'].str[-3:] # cleaning up clone id names
meta_bulks.head()

,Project_ID,Donor,Number_Of_Cells,Clone_ID
Sample_ID,,,,
P9855_2001,P9855,YFV2003,25,A03
P9855_2002,P9855,YFV2003,25,A03
P9855_2003,P9855,YFV2003,25,A03
P9855_2004,P9855,YFV2003,25,A05
P9855_2005,P9855,YFV2003,25,A05


In [6]:
meta_bulks.shape

(69, 4)

In [7]:
meta_bulks['Clone_ID'].unique()

array(['A03', 'A05', 'A12', 'D09', 'D12', 'F10', 'H01', 'H10', 'A08',
       'B10', 'C11', 'F02', 'F04', 'A04', 'B11', 'F01', 'G01', 'G09',
       'H04', 'H05'], dtype=object)

The sequencing data (in merged_gene_counts.txt) contains a separate row for each  gene.  The name of each column contains list of filenames (I'm guessing input for star), we will clean up column names.

In [102]:
read_merged_genes = pd.read_csv('merged_gene_counts.txt', sep='\t', index_col=0)
read_merged_genes.columns = read_merged_genes.columns.str[:10] # 10 characters in P9855_20**
#read_merged_genes.drop(['gene_name'],axis=1, inplace=True)
#read_merged_genes = read_merged_genes.transpose() # tanspose dataframe for later analysis infrastructure
#read_merged_genes.index.names = ['Sample_ID']
read_merged_genes.head()

,gene_name,P9855_2019,P9855_2054,P9855_2072,P9855_2025,P9855_2076,P9855_2068,P9855_2006,P9855_2078,P9855_2042,...,P9855_2009,P9855_2016,P9855_2061,P9855_2015,P9855_2062,P9855_2012,P9855_2080,P9855_2070,P9855_2030,P9855_2031
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000223972,DDX11L1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000227232,WASH7P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000243485,MIR1302-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000237613,FAM138A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000268020,OR4G4P,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
read_merged_genes.to_pickle('P9855_rawreads.pkl') # Save file as a pickle.
meta_bulks.to_pickle('P9855_meta.pkl') # Pickle the metadata too.

The next function loads an entire *list* of cells, and places their gene expression data into the rows of a matrix.  The rows are indexed by the cell names, and the columns by genes.  The data is the gene expression, as raw number of reads.  This may take a little while, so we give progress updates every 10 cells.

# 2.  Munging data

Start here if you have the pickles!  We filter the data a bit, before normalization downstream.

In [92]:
bulks_raw = pd.read_pickle('P9855_rawreads.pkl') # Load bulks expression matrix from a pickle.
meta_df = pd.read_pickle('P9855_meta.pkl') # Load metadata from a pickle.

In [93]:
genes = list(bulks_raw.columns)  # The names of the genes. 
bulks = list(bulks_raw.index) # The names of the bulks.
clones = sorted(list(meta_bulks.Clone_ID.unique())) # The names of the clones.

## Removing TCRs and rarely-expressed genes

T cells have special genetically rearranged receptors called TCRs.  These are made of segments called TRBV9, TRBJ2-4, TRAV12-2, TRAJ14, etc.  Bascally any gene that is called these letters followed by a number -- TRBV, TRBJ, TRAV, TRAJ -- is part of this receptor and they are defined as being clonal.  Therefore we exclude these genes since we want to find more interesting similarities within clonal populations.

The following loads a list of genes to be excluded from the data for later analysis.  The excluded genes should be given in a csv file with *one* column.  No row labels should be given.  The first row should be a descriptive header, like "Genes to exclude."

In [126]:
exc_filename = 'TRgenes.csv'  # CHANGE this if needed.  I added TRAC and TRDV3 as requested.
exc_df = pd.read_csv(exc_filename, sep=',', header=0)
exclude_genes = exc_df.iloc[:,0].tolist()

In [134]:
ensembl_hgnc_meta_filename = 'All_Gene_Info_Metadata.csv'
ensembl_hgnc_meta_df = pd.read_csv(ensembl_hgnc_meta_filename, sep=',', header=0)
ensembl_gene_id = ensembl_hgnc_meta_df.iloc[:,:2].to_dict(orient='records')
#exclude_ensembl = [ensembl_id for gene in ensembl_gene_id if gene in exclude_genes]

In [135]:
ensembl_gene_id

[{'ensembl_gene_id': 'ENSG00000261657', 'hgnc_symbol': 'SLC25A26'},
 {'ensembl_gene_id': 'ENSG00000261657', 'hgnc_symbol': 'SLC25A26'},
 {'ensembl_gene_id': 'ENSG00000261657', 'hgnc_symbol': 'SLC25A26'},
 {'ensembl_gene_id': 'ENSG00000223116', 'hgnc_symbol': nan},
 {'ensembl_gene_id': 'ENSG00000233440', 'hgnc_symbol': 'HMGA1P6'},
 {'ensembl_gene_id': 'ENSG00000207157', 'hgnc_symbol': 'RNY3P4'},
 {'ensembl_gene_id': 'ENSG00000229483', 'hgnc_symbol': 'LINC00362'},
 {'ensembl_gene_id': 'ENSG00000252952', 'hgnc_symbol': 'RNU6-58P'},
 {'ensembl_gene_id': 'ENSG00000235205', 'hgnc_symbol': 'TATDN2P3'},
 {'ensembl_gene_id': 'ENSG00000232849', 'hgnc_symbol': 'LINC00363'},
 {'ensembl_gene_id': 'ENSG00000236803', 'hgnc_symbol': 'SDAD1P4'},
 {'ensembl_gene_id': 'ENSG00000232163', 'hgnc_symbol': 'RPLP1P13'},
 {'ensembl_gene_id': 'ENSG00000252365', 'hgnc_symbol': nan},
 {'ensembl_gene_id': 'ENSG00000229558', 'hgnc_symbol': 'SACS-AS1'},
 {'ensembl_gene_id': 'ENSG00000235784', 'hgnc_symbol': 'HNRNPA1P

In [95]:
def get_relevant(gf, eg, prevalence=0.1, threshold = 10): # gf = expression matrix, eg = list of genes to exclude
    '''
    Outputs True if the gene is relevant for analysis.  We throw out excluded genes.
    By default, we take genes that are found in at least 5% of all cells at a level of
    10 counts or more.
    '''
    nonzero_count = (gf > threshold).sum(axis=0)
    nonzero_proportion = nonzero_count / len(gf)
    return [gene for gene in gf.columns if
           (gene not in eg) & 
           (nonzero_proportion[gene] > prevalence)]

In [65]:
bulks_raw.columns.get_loc('C1orf174'), bulks_raw.columns.get_loc('LRRC47')

(209, 203)

In [69]:
bulks_raw.sort_index().iloc[28:34,203:213]

gene_name,LRRC47,RN7SL574P,RP1-286D6.5,CEP104,DFFB,AL691523.1,C1orf174,LINC01134,RP13-614K11.1,RP13-614K11.2
P9855_2031,103,0,0,0,0,0,190,0,0,0
P9855_2032,0,0,0,0,0,0,171,0,0,0
P9855_2033,0,0,0,0,0,0,50,0,0,0
P9855_2034,0,0,0,2,0,0,232,0,0,0
P9855_2035,66,0,0,0,0,0,169,0,0,0
P9855_2036,76,0,0,0,0,0,204,0,0,0


In [88]:

genes_relevant = get_relevant(bulks_raw, exclude_genes)


In [96]:
#print("{} cells are measured, from {} to {}.".format(len(cells), cells[0], cells[-1]))
print("{} bulks are measured, from {} to {}.".format(len(bulks), bulks[0], bulks[-1]))
print("{} genes are measured, from {} to {}.".format(len(genes),genes[0],genes[-1]))
genes_excluded = [gene for gene in exclude_genes if gene in genes]
print("{} TCR genes were excluded, from {} to {}.".format(len(genes_excluded), genes_excluded[0], genes_excluded[-1]))
print("{} genes are considered relevant, from {} to {}.".format(len(genes_relevant), genes_relevant[0], genes_relevant[-1]))

73 bulks are measured, from P9855_2019 to P9855_2031.
63677 genes are measured, from ENSG00000223972 to ENSG00000235857.


IndexError: list index out of range

## Removing poor quality libraries and overexpressing cells

Next we remove poor quality libraries -- by removing cells which express too few genes.  We also remove cells that may be dividing, and thus express too many genes.  To visualize this, we produce "violin-plots" giving the number of genes expressed by each cell, sorted by clonality.

In [ ]:
def nGene(ge, md, cutoff = 10, plot=True): # ge = gene expression, md = meta data
    nG = ge.apply(lambda row: sum(row > cutoff), axis=1) # Number of genes expressed.
    nG.name = 'num_genes'
    clonalities = md.Clone_ID
    nGene_df = pd.concat([nG, clonalities], axis=1)
    if plot:
        fig,ax = plt.subplots(figsize=(12,8))
        sns.violinplot(x="Clone_ID", y="num_genes", inner='quartiles', data=nGene_df, ax=ax)
        sns.swarmplot(x="Clone_ID", y="num_genes", color="white", size=3, data=nGene_df, ax=ax);
    return nGene_df

In [ ]:
nG = nGene(bulks_raw, meta_df)

Now we pick out the "good cells", whose expressed-gene-number is within two standard deviations of the mean (within each clone).

In [ ]:
nG_std = nG.groupby('Clone_ID').apply(np.std)
nG_means = nG.groupby('Clone_ID').apply(np.mean)
nG_lowcut = (nG_means - 2*nG_std).num_genes
nG_highcut = (nG_means + 2*nG_std).num_genes
cells_good = [c for c in cells if 
              (nG.num_genes[c] >= nG_lowcut[nG.Clone_ID[c]]) and 
              (nG.num_genes[c] <= nG_highcut[nG.Clone_ID[c]]) 
             ]
print('{} cells remaining after {} poor libraries removed.'.format(len(cells_good), len(cells) - len(cells_good)))

In [ ]:
nG = nGene(bulks_raw.loc[cells_good], meta_df.loc[cells_good]) # Post-trimming violin-plot.

#  3.  MRN Normalization.

Here we implement MRN Normalization on the gene expression data, closely following the convenient outline in Section 3.2 of [Maza](https://www.frontiersin.org/articles/10.3389/fgene.2016.00164/full).  We begin by putting our filtered data into a dataframe.  The dataframe `EM` (for "expression matrix") contains the gene expressions for the cells, filtered as above.

In [ ]:
EM = bulks_raw#[genes_relevant]

In [ ]:
EM.head()

## Step I:  Prenormalization by library size.

In Maza's article, $X_{gkr}$ stands for the raw count (number of reads) of gene $g$, for a cell number $r$ among clone $k$.  This information is contained in our expression matrix `EM`.  The first step is to normalize by library size, dividing $X_{gkr}$ by the total number of reads $N_{kr}$ of cell with numbers $k$, $r$.  

We find the total number of reads for each cell by simply summing the numbers in each row of the data frame `EM`.  We don't need to worry about the separate indices $k$ and $r$ yet.  We examine the resulting "library size" below.

In [ ]:
library_size = EM.sum(axis=1) # Drop the clone column.  Sum along rows.
print(library_size.head())
library_size.describe()

The library size is about 1.6 million +/- 440,000.  Now we normalize the expression matrix by dividing every cell's raw counts by the cell's library size.  Note that `EM` is a dataframe whose rows are indexed by the cells.  `library_size` is a series (basically an array) whose rows are indexed by cells.  Numpy/pandas will divide one array by another, term by term, if they have the same size.  So it can divide *each column* of `EM` by `library size`, in a quickly-broadcasted division.  To perform this on every column, we use the `apply` method with a "lambda" function... it's the quickest method I know.

In [ ]:
Y = EM.apply(lambda column : column / library_size)
Y

In [ ]:
Y.sort_index().iloc[28:31, 68:78] #just looking for some nonzero values in the df

Now `Y` is the dataframe with counts normalized by library size, and we pass to the next step.

## Step II:  Creation of reference sample.

A difference between TMM (used in edgeR), RLE (used in DeSeq2), and MRN, is how they create a reference sample.  In MRN, a reference sample is created by averaging the previous dataframe `Y` over cells within a single condition (clone).  We carry this out here.

Note the method-chaining in defining `Y_clonal` below.  The `assign` method tacks on a new column to `Y` for the clone_IDs.  The `groupby` method then groups cells by clonotypes.  The `apply` method then takes the means within each clonotype.  The end result is to replace the (prenormalized) gene expression for *each cell* by the averages for each clonotype.  

In [ ]:
Y_clonal = Y.assign(Clone=meta_df['Clone_ID']).groupby('Clone').apply(np.mean) # Takes a few seconds.
display(Y_clonal)

Let's take our reference sample to be the A7 clonal average.

In [ ]:
Y_ref = Y_clonal.loc['D12'] # Our reference sample.  Basically an average of all cells of clonotype D12.
Y_ref.describe()

## Step III:  Computation of size relative to reference sample.

Now we compute relative scaling factors for each clone, based on the median fold-changes between gene expression within that clone and the reference clone (A7, we have chosen above).  Due to dropouts (values of 0 in gene expression), we discard genes with zeros when computing fold changes.  This avoids division-by-zero problems, and it matches the implementation of [Maza et al.](https://www.tandfonline.com/doi/full/10.4161/cib.25849).  (See the R code in the supplementary information).  

In [ ]:
tau = pd.Series(index = clones)
for clone in clones: # Why not a little loop.
    numerator = Y_clonal.loc[clone]
    denominator = Y_ref
    ok_genes = [gene for gene in genes_relevant if (numerator[gene] != 0) and (denominator[gene] != 0)]
    ratios = numerator[ok_genes] / denominator[ok_genes] # The ratios.
    tau[clone] = np.median(ratios)
print(tau)

## Step IV:  Adjustment of relative scaling factors.

This step does not occur in MRN normalization.  Yay!

## Step V:  Effective library size.

In [ ]:
# intermediate series to match the clone IDs with the samples
clonalities = pd.Series(meta_df['Clone_ID'], index=library_size.index)
clonalities.head()

In [ ]:
#intermediate series to map tau scaling values to the samples
scalar_lib_size = clonalities.map(tau)
scalar_lib_size.head()

In [ ]:
#final series, for this step at least as the library_size values are scaled by tau values
effective_lib_size = scalar_lib_size * library_size
effective_lib_size.head()

## Step VI:  Computation of relative normalization/size factors.

In [ ]:
geo_mean = np.exp(effective_lib_size.apply(np.log).mean()) # The geometric mean of the effective_lib_size series.
geo_mean # A single number!

In [ ]:
effective_lib_size.mean() # Compare the geometric mean (above) to the usual mean here.  They shouldn't be *too* far.

In [ ]:
relative_normalization_factor = effective_lib_size / geo_mean
relative_normalization_factor.head()

## Step VII:  Normalization of counts.

In [ ]:
# here we normalize our raw counts by the relative normalization factor we just
# calculated in Step VI
mrn_counts = EM.div(relative_normalization_factor, axis=0)
mrn_counts.head()

# 4.  Quick shot at machine learning

After preprocessing the data, one might want to quickly check how well gene expression can predict clonotype.  The following steps will test out a linear support vector classifier. 

In [ ]:
import statsmodels.api as sm

# Import various functions for stats.
from scipy.stats import f_oneway, binom, binom_test, kruskal, norm # For p-values, kruskal-wallis, Gaussian
from scipy.stats import kurtosis, skew # For detecting bimodality
from statsmodels.formula.api import ols

from bisect import bisect # See bisect.bisect.
from itertools import compress # I don't recall why this is here.

# Import from scikit, for machine learning.
# from sklearn import mixture  # Import Gaussian mixture

from sklearn.preprocessing import Binarizer, MinMaxScaler

from sklearn.model_selection import ShuffleSplit, train_test_split, GridSearchCV

from sklearn.feature_selection import SelectKBest, chi2, f_classif

from sklearn.metrics import accuracy_score

from sklearn.svm import LinearSVC # Linear Support Vector Classifier 
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
 
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin

from sklearn.manifold import TSNE


In [ ]:
X = mrn_counts
X_cells = mrn_counts.index
y = meta_df.loc[X_cells].Clone_ID

In [ ]:
SVMpipe = make_pipeline(MinMaxScaler(), 
                        SelectKBest(score_func = f_classif, k=100), 
                        LinearSVC(random_state=0, tol=1e-4, max_iter = 2000))

In [ ]:
# DANGER DANGER:  The following suppresses all warnings from Python.
# Otherwise, you'll probably get a bunch of convergence warnings...
# Alternatively, increase the number of iterations in the SVM.

import warnings
warnings.filterwarnings('ignore') # Ignore warnings.

param_grid = dict(linearsvc__C=[0.001, 0.01, 0.1, 1.0, 10, 100],
                  selectkbest__k = range(50,251,25),
                 )

grid_search = GridSearchCV(SVMpipe, param_grid=param_grid, cv=5, iid=False) #5-fold cross-validation.
grid_search.fit(X,y)

In [ ]:
def CV_map(gs, swap=False):  # parameter_grid and grid_search.
    '''
    A general-purpose visualization for grid-search cross-validations.
    Displays predictive accuracy as a heatmap, based on different
    choices of parameters.  Displays optimal parameters in title.
    
    Args:
        gs (sklearn.model_selection.GridSearchCV):  The grid-search object.
        swap:  Set to True to switch the axes.
        
    Returns:
        None
        
    '''
    results = gs.cv_results_
    means = results['mean_test_score']
    params = list(gs.param_grid.keys())
    
    ser = pd.Series(list(means),
                  index=pd.MultiIndex.from_tuples(
                      [tuple(d.values()) for d in results['params']])
               )
    gs_df = ser.unstack().fillna(0)
    gs_df.shape 
    if swap:
        gs_df = gs_df.transpose()
        params = params[::-1] # Switch the order of the parameter list.
    
    f, ax = plt.subplots(figsize=(12,12))
    sns.heatmap(gs_df, annot=True, square=True, fmt="2.0%", linewidths=.5, 
                cbar_kws={'fraction':0.025, 'pad':0.05}, ax=ax)
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([.5, .6, .7, .8, .9])
    cbar.set_ticklabels(['50%', '60%', '70%', '80%', '90%'])
    ax.set_xlabel(params[1])
    ax.set_ylabel(params[0])
    plt.yticks(rotation=0)
    bestacc = gs.best_score_
    ax.set_title("Result of grid-search with 5-fold cross-validation. \n"
                 "Best score {0:2.1%} with parameters \n"
                 "{1} ".format(gs.best_score_, gs.best_params_))

In [ ]:
CV_map(grid_search)

In [ ]:
from sklearn import base

def confusion_matrix(X,y,pipe, reps=2, classlist = None):
    if classlist:
        classes = classlist
    else:
        classes = sorted(y.unique())
    counts = pd.DataFrame(0, index=classes, columns=classes)
    cpops = pd.DataFrame(0, index=classes, columns=['pop'])
    correct = 0
    total = 0
    for t in range(reps):
        this_pipe = base.clone(pipe)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        this_pipe.fit(X_train, y_train)
        y_pred = this_pipe.predict(X_test)
        actual = y_test
        predicted = y_pred
        for j in range(len(y_test)):
            total += 1
            if y_test[j] == y_pred[j]:
                correct += 1
            cpops.loc[y_test[j],'pop'] += 1
            counts.loc[y_test[j], y_pred[j]] += 1
    
    valid = counts.apply(lambda column : column/cpops['pop']) #['pop'] )
    
    f, ax = plt.subplots(figsize=(len(classes)+1,len(classes)))
    sns.heatmap(valid, annot=valid, square=True, fmt="2.0%", linewidths=.5, 
                cbar_kws={'fraction':0.046, 'pad':0.04}, ax=ax)
    cbar = ax.collections[0].colorbar
    cbar.set_ticks([0, .25, .5, .75, 1])
    cbar.set_ticklabels(['0%', '25%', '50%', '75%', '100%'])
    ax.set_ylabel("Actual class")
    ax.set_xlabel("Predicted class")
    ax.set_title("Overall prediction accuracy {:0.3%}. \n"
                 "{} trials, in {} batches with independent splits.".format(correct/total, total, reps))

In [ ]:
opt_k = grid_search.best_params_['selectkbest__k'] # Use optimal number of genes.
opt_C = grid_search.best_params_['linearsvc__C'] # Use optimal C-parameter for linear SVM.

SVMpipe = make_pipeline(MinMaxScaler(), 
                        SelectKBest(score_func = f_classif, k=opt_k), 
                        LinearSVC(random_state=0, tol=1e-4, max_iter = 2000, C=opt_C))

confusion_matrix(X,y,SVMpipe, reps=50, classlist = clones)